In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import onnx
from onnx_tf.backend import prepare

ModuleNotFoundError: No module named 'onnx'

In [3]:
class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} \tLoss: {:.6f}'.format(
                    epoch,  loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the maxlog-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
        

In [ ]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])), batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False, 
                        transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])), batch_size=1000, shuffle=True)

In [ ]:
device = torch.device("cuda")
print(device)
model_torch = Net().to(device)
optimizer = optim.SGD(model_torch.parameters(), lr=0.01, momentum=0.5)
for epoch in range(10):
    train(model_torch, device, train_loader, optimizer, epoch)
    test(model_torch, device, test_loader) 

In [ ]:
torch.save(model_torch.state_dict(), 'mnist.pth')

In [ ]:
model_torch = Net()
model_torch.load_state_dict(torch.load('mnist.pth'))
dummy_input = Variable(torch.randn(1, 1, 28, 28))
torch.onnx.export(model_torch.eval(), dummy_input, "mnist.onnx")

In [ ]:
model_onnx = onnx.load('mnist.onnx')
model_tf = prepare(model_onnx)

In [ ]:
import numpy as np
from IPython.display import display
from PIL import Image
import matplotlib.pyplot as plt

print('Image 1:')
img = Image.open('two.png').resize((28, 28)).convert('L')
display(img)
temp = np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :]
print(temp.shape)
output = model_torch(torch.from_numpy(temp))
print('The digit is classified as', output.detach().numpy(), 'pytorch')
output = model_tf.run(temp)
print('The digit is classified as', output, 'tf')

In [ ]:
print('Image 2:')
img = Image.open('three.png').resize((28, 28)).convert('L')
display(img)
temp = np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :]
print(temp.shape)
output = model_torch(torch.from_numpy(temp))
print('The digit is classified as', output.detach().numpy(), 'pytorch')
output = model_tf.run(temp)
print('The digit is classified as', output, 'tf')

In [ ]:
model_tf.export_graph('mnist.pb')

In [ ]:
for param in model_torch.parameters():
    print(param.data)
    print(param.data.shape)

In [ ]:
import tensorflow as tf
imported = tf.saved_model.load('mnist.pb')
f = imported.signatures["serving_default"]
#print(f(x=tf.constant([[1.]])))

In [ ]:
f(tf.convert_to_tensor(temp))

In [ ]:
model_new = tf.keras.models.load_model('mnist.pb')
f_new = model_new.signatures["serving_default"]

In [ ]:
f_new(tf.convert_to_tensor(temp))

In [ ]:
print(type(f_new.inputs[0]))

In [4]:
import torch
from torch import nn
cuda = torch.cuda.is_available()
print(cuda)

True


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(cuda)
print(model)

In [ ]:
x = torch.randn(1, 10)

class MyModel(nn.Module):
    def __init__(self, inplace=False):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(10, 10)
        self.fc1 = nn.Linear(10, 1)
        self.relu = nn.ReLU(inplace=inplace)
        
    def forward(self, x):
        x = self.relu(self.fc1(self.fc(x)))
        return x

model = MyModel()

In [ ]:


model.eval()
# out of place
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
d = dict(model.named_modules())

for i in d:
    print(i)
    model.fc1.register_forward_hook(get_activation(i))

model.fc.register_forward_hook(get_activation('fc'))
output = model(x)
activation['fc']

In [ ]:
model.eval()
hooks = {}
def get_activation(name):
    def hook(model, input, output):
        hooks[name] = output.detach()
    return hook

for name, module in model.named_modules():
    print(name)
    hooks[name] = module.register_forward_hook(get_activation(name))
    #output = model(x)
    print(hooks[name])
lists = model.named_modules()
print(lists[2][0])


In [ ]:
output = model(x)
print(hooks['fc'])
output = model(x)
print(hooks['fc1'])